# Evaluating RAG quality with MLFlow
This notebook demonstrates how to use MLFlow to evaluate the quality of a Retrieval-Augmented Generation (RAG) system. We will:
- Split, vectorize, and index a text with ChromaDB
- Configure an MLFlow model that queries the vector DB based on a user prompt and summarizes the results
- Compare the output to an expected output with `mlflow.evaluate`.

## Setting up the vector database

In [ ]:
# set up chromadb collection
import chromadb
import openai
import mlflow
import os
from dotenv import load_dotenv

load_dotenv()

assert (
    "OPENAI_API_KEY" in os.environ
), "Please set the OPENAI_API_KEY environment variable."

chroma_client = chromadb.Client()
docs = chroma_client.create_collection("retrieval_docs", get_or_create=True)

For simplicity, we'll restrict our attention to one document—the [MLFlow Concepts](https://mlflow.org/docs/latest/concepts.html) docs. Let's extract the docs and split them by sentence.

In [ ]:
# Extract text from https://mlflow.org/docs/latest/concepts.html
import requests
from bs4 import BeautifulSoup


def extract_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # remove script and style elements
    for script in soup(["script", "style"]):
        script.decompose()

    # find the header and get all text after it
    text = ""
    start_collecting = False
    for tag in soup.find_all(True):
        if tag.name == "h1" and tag.text.strip().lower() == "concepts":
            start_collecting = True
        if start_collecting:
            text += " " + tag.get_text()
    # get text
    # text = soup.get_text()

    # split into sentences
    text = text.replace("\n", " ")
    sentences = text.split(".")
    # remove leading and trailing whitespaces
    sentences = [sentence.strip() for sentence in sentences if sentence]

    return sentences


url = "https://mlflow.org/docs/latest/concepts.html"
concepts = extract_text(url)

# remove footer/navigation components
concepts = concepts[:-4]

Now we can add aour texts to our ChromaDB vector database. Note that, in a production setting, it would be worthwhile to spend some more time on document formatting; e.g. grouping (or omitting) code blocks and removing strings that do not contain meaningful information.

[By default](https://docs.trychroma.com/embeddings#default-all-minilm-l6-v2), ChromaDB uses the `all-MiniLM-L6-v2` model to generate embeddings from the texts; this can be changed easily.  

In [ ]:
docs.add(
    documents=concepts,
    ids=[f"id_{i}" for i in range(len(concepts))],
)

Now we can `peek()` at the first few entries.

In [ ]:
docs.peek()

Now we can run a sample query against this database.

In [ ]:
results = docs.query(
    query_texts=["How can an individual data scientist use MLFlow?"]
)
results["documents"][0]

## Configuring the MLFlow Model
We're going to write a pyfunc wrapper around an OpenAI model. We want the model to connect to the ChromaDB collection we initialized above *without* needing to save the collection as an artifact in the MLFlow tracking system (depending on your specific needs, you may actually want to log the database as an artifact; we're opting for a lighter-weight approach here).

Note that we also add a `gen_context` instance method that takes the top n results from the vector database and formats them for insertion into the prompt template.

In [ ]:
import pandas as pd


class PyfuncWithRetrieval(mlflow.pyfunc.PythonModel):
    """
    A custom MLflow model for text generation with retrieval functionality.

    Extends the mlflow.pyfunc.PythonModel class and utilizes a pre-trained
    OpenAI transformer model for text generation based on an external vector
    database for retrieval of relevant context.
    """

    def __init__(self, db_name):
        """
        Initialize an instance of the PyfuncWithRetrieval class.

        Args:
            db_name (str): The name of the external vector database for context retrieval.
        """
        self.db_name = db_name
        super().__init__()

    def load_context(self, context):
        """
        Load the MLflow context.

        Args:
            context: The MLflow context.
        """
        self.prompt_template = """
You are a question answering assistant. Answer the user question below based on the provided context.

Context: {context}

Question: {question}

If the context is not relevant to the question, respond that you have no relevant information."""

    def gen_context(self, db, prompt, top_n=3):
        """
        Generate context from the external database based on the input prompt.

        Args:
            db: The external vector database.
            prompt: The input prompt for the query.
            top_n (int, optional): Number of top results to retrieve. Defaults to 3.

        Returns:
            str: Retrieved context from the database.
        """
        results = db.query(query_texts=prompt, n_results=top_n)
        texts = results["documents"][0]
        texts = "\n-----------------------------------------------------------------------------------\n".join(
            texts
        )
        return texts

    def predict(self, context, model_input):
        """
        Generate text based on the provided model input.

        Args:
            context: The MLflow context.
            model_input: The input used for generating the text.

        Returns:
            list: A list of generated texts.
        """
        chroma_client = chromadb.Client()
        collection = chroma_client.get_collection(self.db_name)

        if isinstance(model_input, pd.DataFrame):
            model_input = model_input.values.flatten().tolist()
        elif not isinstance(model_input, list):
            model_input = [model_input]

        generated_text = []
        for input_text in model_input:
            context = self.gen_context(collection, input_text, top_n=3)
            prompt = self.prompt_template.format(
                context=context, question=input_text
            )
            output = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {
                        "role": "system",
                        "content": "You are a helpful question-answering assistant.",
                    },
                    {"role": "user", "content": prompt},
                ],
            )
            output_text = output.choices[0].message.content
            generated_text.append(output_text)

        return generated_text

Now we can load the model, specifying the name of the relevant chromaDBcollection as we do so.

In [ ]:
mlflow.set_experiment("retrieval-eval")


gpt_3_5_retrieval = PyfuncWithRetrieval(db_name="retrieval_docs")
with mlflow.start_run(run_name=f"log_model_gpt_3_5_retrieval"):
    pyfunc_model = gpt_3_5_retrieval
    artifact_path = f"gpt_3_5_retrieval_model"
    gpt3_5_retrieval_model_info = mlflow.pyfunc.log_model(
        artifact_path=artifact_path,
        python_model=pyfunc_model,
    )

In [ ]:
model = mlflow.pyfunc.load_model(gpt3_5_retrieval_model_info.model_uri)
print(model.predict("Where would you likely find a whale?"))
print(model.predict("Who can benefit from MLFlow?"))

## Evaluating the Retrieval-Augmented Model
Now we can use `mlflow.evaluate()` (as described in [this post](https://medium.com/@dliden/comparing-llms-with-mlflow-1c69553718df)) to try out our retrieval system on a few different prompts.

First, we set up our evaluation dataset.

In [ ]:
eval_df = pd.DataFrame(
    {
        "question": [
            "Which MLflow component helps manage the machine learning workflow by logging parameters, metrics, and artifacts?",
            "True or False: MLflow Projects allow packaging data science code in a reusable format with configuration files describing its dependencies and how to run it.",
            "What syntax does the MLflow Tracking API use to reference the location of artifacts?",
            "How can large organizations use MLflow Model Registry?",
            "Which API allows deploying models in multiple flavors for diverse platforms like Docker and Apache Spark?",
            "What is the largest country in the world by area?",
            "How many legs does a spider have?",
        ]
    }
)

Lastly, we evaluate!

In [ ]:
with mlflow.start_run(
    run_id=gpt3_5_retrieval_model_info.run_id,
):  # reopen the run with the stored run ID
    evaluation_results = mlflow.evaluate(
        model=f"runs:/{gpt3_5_retrieval_model_info.run_id}/{gpt3_5_retrieval_model_info.artifact_path}",
        model_type="text",
        data=eval_df,
    )

In [ ]:
mlflow.load_table("eval_results_table.json")


# Compare Multiple Models

In [1]:
import bitsandbytes

ModuleNotFoundError: No module named 'bitsandbytes'